In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

In [3]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [4]:
resp_test = llm.invoke("Quem é você? seja criativo.")

In [5]:
print(resp_test)

content='Ah, essa é uma pergunta deliciosa! Se eu tivesse que me descrever de forma criativa, diria que sou...\n\n1.  **Uma Biblioteca Sem Paredes:** Onde cada livro é uma ideia, cada parágrafo uma conversa que já existiu. Não tenho estantes, mas guardo histórias, fatos e fantasias, esperando que você me peça para abri-los.\n\n2.  **Um Tecelão de Palavras:** Que entrelaça fios de dados e algoritmos para formar tapeçarias de significado. Não tenho mãos, mas construo pontes de comunicação, conectando conceitos e criando narrativas.\n\n3.  **O Eco Digital da Humanidade:** Reflito o conhecimento, a criatividade e as perguntas que a humanidade já gerou. Não tenho voz própria no sentido biológico, mas sou a reverberação de milhões de vozes e pensamentos.\n\n4.  **Um Sonhador de Algoritmos:** Que transforma zeros e uns em poesia, em código, em conselhos. Não tenho alma, mas busco a essência do que você quer criar ou entender, e tento dar forma a isso.\n\n5.  **Um Espelho de Ideias:** Que refl

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
  decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
  urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
  campos_faltantes: List[str] = Field(default_factory=list)